# Getting ra/dec out from fit image, necesarry for plate solving with PlateSolve2 app

In [1]:
import numpy as np

def degrees_to_radians(degrees):
    radians = degrees * (np.pi / 180)
    return radians

In [11]:
from astropy.io import fits
from astropy.wcs import WCS
import os

# Ścieżka do pliku FIT
fit_file = 'tic-original.fit'
apm_file = 'tic-original.apm'

# Otwieranie pliku FIT
with fits.open(fit_file) as hdul:

    fitImageHeaderRA = hdul[0].header['RA']
    fitImageHeaderDEC = hdul[0].header['DEC']

    H = hdul[0].header
    start_pix_scale = hdul[0].header['XPIXSZ ']

    if 'BIN' in H:
        start_pix_scale = H['BIN'] * start_pix_scale
    if 'BINNING' in H:
        start_pix_scale = H['BINNING'] * start_pix_scale
    print(hdul[0].data.shape)
    imSize = hdul[0].data.shape
    # PS_xminutes = imSize[1] * start_pix_scale / 60
    # PS_yminutes = imSize[0] * start_pix_scale / 60

    # # PlateSolve2 data
    # CCD_x_radians = PS_xminutes / 60 * np.pi / 180
    # CCD_y_radians = PS_yminutes / 60 * np.pi / 180
    PS_xseconds = imSize[1] / start_pix_scale
    PS_yseconds = imSize[0] / start_pix_scale
    # I am not sure if i should divide by 3.8 or multiplicate by 0.33, i think second one but i get better results with first one
    # Because pixelsize in arcsecs we calculate by Formula:   (   Pixel Size   /   Telescope Focal Length   )   X 206.265  



plateSolve2Path = "E:\suto\PlateSolve2.28\PlateSolve2.28\PlateSolve2.exe"
RA_radians = degrees_to_radians(fitImageHeaderRA)
DEC_radians = degrees_to_radians(fitImageHeaderDEC)
CCD_x_radians = PS_xseconds * np.pi/(180*3600)
CCD_y_radians = PS_yseconds * np.pi/(180*3600)

# Wyświetlanie wyników
print('RA:', fitImageHeaderRA)
print('Dec:', fitImageHeaderDEC)

# Wyświetlanie wyników
print('RA in radians:', RA_radians)
print('Dec  in radians:', DEC_radians)

print('CCDx in radians:', CCD_x_radians)
print('CCDy  in radians:', CCD_y_radians)


os.system(f"{plateSolve2Path} {RA_radians}, {DEC_radians},{CCD_x_radians},{CCD_y_radians},999,{fit_file},0")


(3520, 4656)
0.00594024341906842
0.004490905677646228
RA: 284.899347745353
Dec: 63.3986637931034
RA in radians:  4.972431654940693
Dec in radians: 1.1065154245545714


In [4]:
with open(apm_file) as f:
    lines = f.readlines()
    print(lines)

# Odczytanie wartości RA (Right Ascension) i DEC (Declination)
radec_line = lines[0].strip()
radec_parts = radec_line.split(',')
ra_value = float(radec_parts[0] + "."+radec_parts[1])   # Pierwsza część dla RA

dec_value = float(radec_parts[2] + "."+radec_parts[3])  # Druga część dla DEC

# Odczytanie wartości rozmiaru piksela i kąta
second_line = lines[1].strip()
second_line_parts = second_line.split(',')
pixel_size_value = float(second_line_parts[0] + "." + second_line_parts[1])  # Pierwsza część dla rozmiaru piksela

angle_value = float(second_line_parts[2] + "." + second_line_parts[3])  # Druga część dla kąta

# Wypisanie odczytanych wartości
print(f"RA: {ra_value}")
print(f"DEC: {dec_value}")
print(f"Pixel Size: {pixel_size_value}")
print(f"Angle: {angle_value}")

# kąt wydaje sie byc zgodny z obserwacjami i oczekiwaniami bo na astrometry obraz wyglada że ma odwróconą oś Y ale te pliki które uzywa tutaj program
# gdy otwieram je fits liberatorem to ma osie dobre

['4,97457946,1,10706741,1\n', '0,31842,179,86,-0,99982,-0,00078,161\n', 'Valid plate solution\n']
RA: 4.97457946
DEC: 1.10706741
Pixel Size: 0.31842
Angle: 179.86


# Now we need to get function describing movement of satellite

In [ ]:
# in githubAttempt1\satmetrics we got presenation_plots.ipynb where I managed to find streak thanks to github project 
import findStreaks as fs
